## Overview

#### This note book is a pipline connecting to the git hub repository: https://github.com/dsfsi/covid19africa and extrating the following data related to the corona virus

1. Number of tests performed daily 
2. The Provincial case breakdown for South Africa

### Licence

\https://github.com/dsfsi/covid19za/blob/master/data/LICENSE.md

#### Import Database Structure
This is the SQL database structure used


In [9]:
from database import model as m

In [8]:
pip install --upgrade git+https://github.com/kopano-m/database.git

  Cloning https://github.com/kopano-m/database.git to c:\users\kopan\appdata\local\temp\pip-req-build-hjk16a4a
  Created wheel for database: filename=database-0.1-py3-none-any.whl size=2139 sha256=ac6371113e9a92b7e6a350f5e879eb1ec0886b3be7b9b5ba18b00edfc38f3661
  Stored in directory: C:\Users\kopan\AppData\Local\Temp\pip-ephem-wheel-cache-n8lr6ccz\wheels\b7\38\cd\891c05ada66982ae41125c0ccd3006f52bc405fe1a6b1ca6e5
Successfully built database
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/kopano-m/database.git 'C:\Users\kopan\AppData\Local\Temp\pip-req-build-hjk16a4a'


#### Connect to database
RDS SQL Server Data Base

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

secret = {
    "username": "admin",
    "password": "",
    "host": "database-1.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com",
    "port": "1433"
}

engine = create_engine(
    'mssql+pymssql://' +
    secret['username'] + ':' + secret['password'] + '@' + secret['host'] + ':' +
    str(secret['port']) + '/Corona'

)

session = sessionmaker()(bind=engine)

#### Get Data

In [11]:
df_tests = pd.read_csv("https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv")
df_provices = pd.read_csv("https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv")

#### Clean data

##### 1. Clean Test Data

In [12]:
df_tests["date"] = df_tests["date"].apply(lambda x:pd.to_datetime(x))

def error(x):
    if pd.isnull(x):
        return 0
    else:
        return x
    
df_tests["cumulative_tests"] = df_tests.cumulative_tests.apply(lambda x: error(x))

##### 2. Clean Provincial Data

In [13]:
df_provices = df_provices.melt(id_vars = ["date","YYYYMMDD"])
df_provices["date"] = df_provices["date"].apply(lambda x: pd.to_datetime(x))

#### Upload data To SQL

##### 1. Upload Test Data

In [15]:
Upload_Country = "South Africa"

for i in range(len(df_tests)):
    
    ## Check Country exisits
    Country = session.query(m.Country).filter(m.Country.country == Upload_Country).first()
    if Country is None:
        Print("Country Not Found")

    
    Tests = m.Tests( 
                        date = df_tests["date"][i],
                        cumulative_tests = int(df_tests["cumulative_tests"][i]),
                        country_id = Country.id
    )
    
    session.add(Tests)

session.commit()
session.close()

OperationalError: (pymssql.OperationalError) (18456, b'DB-Lib error message 20018, severity 14:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20002, severity 9:\nAdaptive Server connection failed (database-1.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com:1433)\n')
(Background on this error at: http://sqlalche.me/e/e3q8)

##### 2. Upload Provincial Data

In [ ]:
Upload_Country = "South Africa"
level = "Provincial"

Upload_Country_id = session.query(m.Country).filter(m.Country.country == Upload_Country).first().id

for i in range(len(df_provices)):
    
    ## Check Country exisits
    Location = session.query(m.Location).filter(m.Location.location == df_provices["variable"][i]).first()
    if Location is None:
        Location = m.Location(country_id = Upload_Country_id,
                              location = df_provices["variable"][i],
                              location_level = level)
        session.add(Location)
        session.commit()

    
    CasesLocal = m.CasesLocal( 
                        date = df_provices["date"][i],
                        confirmed = int(df_provices["value"][i]),
                        location_id = Location.id
    )

    session.add(CasesLocal)
    print(i)

session.commit()
session.close()